In [1]:
!git clone https://github.com/peiro98/ml-domain-adaptation.git
!cp -r ./ml-domain-adaptation/PACS ./PACS
!cp ./ml-domain-adaptation/DANN.py ./DANN.py
!cp ./ml-domain-adaptation/__init__.py ./__init__.py
!rm -rf ./ml-domain-adaptation

!git clone https://github.com/MachineLearning2020/Homework3-PACS
!mv Homework3-PACS/PACS ./PACS/data
!rm -rf Homework3-PACS/

Cloning into 'ml-domain-adaptation'...
remote: Enumerating objects: 14, done.
remote: Total 14 (delta 0), reused 0 (delta 0), pack-reused 14
Unpacking objects: 100% (14/14), done.
Cloning into 'Homework3-PACS'...
remote: Enumerating objects: 10032, done.
remote: Total 10032 (delta 0), reused 0 (delta 0), pack-reused 10032
Receiving objects: 100% (10032/10032), 174.13 MiB | 20.77 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Checking out files: 100% (9993/9993), done.


In [2]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 140 kB 43.4 MB/s 
     |████████████████████████████████| 180 kB 45.6 MB/s 
     |████████████████████████████████| 97 kB 6.9 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=2a56deb5ffd6af9d96cc9d10792aafeb2af291b24bd73fe40eb14f358612df38
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=ccda2f6a426b75027b5e7892997ec801c8d0dc4b0b29b99501e1edd42d0f2556
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


**Import libraries**

In [46]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

from torchvision import transforms

from tqdm import tqdm

from DANN import build_model as build_DANN

In [47]:
import wandb
wandb.init(project="aml-lab03", entity="peiro98", name="aml-lab03")

**Set Arguments**

In [48]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"  # 'cuda' or 'cpu'

# available classes: "dog", "elephant", "giraffe", "guitar", "horse", "house", "person"
NUM_CLASSES = 7

BATCH_SIZE = 64
LR = 5e-5
MOMENTUM = 0.9
WEIGHT_DECAY = 5e-5

NUM_EPOCHS = 50
STEP_SIZE = 20
GAMMA = 0.1

LOG_FREQUENCY = 10
TRAIN_RATIO = 0.75

wandb.config.update({
    "batch-size": BATCH_SIZE,
    "learning-rate": LR,
    "momentum": MOMENTUM,
    "weight_decay": WEIGHT_DECAY,
    "num_epochs": NUM_EPOCHS,
    "step_size": STEP_SIZE,
    "gamma": GAMMA
})

**Define Data Preprocessing**

In [49]:
# Define transforms for training phase
train_transform = transforms.Compose(
    [
        # 227x227 -> 224x224
        transforms.CenterCrop(224),
        # convert to tensor
        transforms.ToTensor(),
        # normalizes tensor with mean and standard deviation
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

# Define transforms for the evaluation phase
eval_transform = train_transform

### Dataset loading

#### Source dataset

In [50]:
DATA_DIR = "PACS/data"
SRC_DATA_DIR = f"{DATA_DIR}/photo/"

from PACS.PACS import PACSDataset

# Prepare Pytorch train/test Datasets
src_dataset = PACSDataset(SRC_DATA_DIR, transform=train_transform)

print("[SRC] train size:", len(src_dataset))

[SRC] train size: 1670


#### Target dataset

In [51]:
TARGET_DATA_DIR = f"{DATA_DIR}/art_painting/"

# Prepare Pytorch train/test Datasets
target_dataset = PACSDataset(TARGET_DATA_DIR, transform=train_transform)

# take the indicies corresponding to train samples
target_train_indices = np.random.choice(
    len(target_dataset), size=int(TRAIN_RATIO * len(target_dataset))
)
target_val_indices = np.setdiff1d(np.arange(len(target_dataset)), target_train_indices)

target_train_dataset = Subset(target_dataset, target_train_indices)
target_validation_dataset = Subset(target_dataset, target_val_indices)

print("[TARGET] train size:", len(target_train_dataset))
print("[TARGET] validation size:", len(target_validation_dataset))

[TARGET] train size: 1536
[TARGET] validation size: 976


**Prepare Dataloaders**

In [52]:
# Train dataloaders
src_dataloader = DataLoader(
    src_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=True
)
# src_train_dataloader = DataLoader(src_train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
# target_train_dataloader = DataLoader(target_train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)

# Validation dataloaders
target_dataloader = DataLoader(
    target_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=True
)
# src_validation_dataloader = DataLoader(src_validation_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=True)
# target_validation_dataloader = DataLoader(target_validation_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=True)

In [53]:
def compute_accuracy(model, dataloader):
    model.train(False)

    # predictions_table = wandb.Table(columns=["image", "label", "preds"])

    running_corrects = 0
    n = 0
    for data, labels in dataloader:
        data = data.to(DEVICE)
        labels = labels.to(DEVICE)

        # Forward Pass
        outputs = model(data, True) # True for classifier

        # Get predictions
        _, preds = torch.max(outputs.data, 1)

        # Update Corrects
        running_corrects += torch.sum(preds == labels.data).data.item()
        n += len(preds)

        # if i == 0:
        #     for image, label, pred in zip(images, labels, preds):
        #         predictions_table.add_data(wandb.Image(image), label, pred)
                
    return running_corrects / n


In [54]:
def compute_classification_loss(model, dataloader):
    criterion = nn.CrossEntropyLoss(reduction='sum')
    model.train(False)
    
    loss, n = 0, 0
    for data, labels in dataloader:
        data = data.to(DEVICE)
        labels = labels.to(DEVICE)

        # Forward Pass
        outputs = model(data, True) # True for classifier

        # compute the loss
        loss += criterion(outputs.detach(), labels)
        n = n + len(labels)
       
    return loss / n


## Train w/ domain adaptation

In [55]:
DANN_da = build_DANN(7, 2, True).to(DEVICE)

# classification => cross entropy loss
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(
    DANN_da.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY
)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

In [56]:
wandb.watch(DANN_da, log_freq=10, )

cudnn.benchmark = True

for epoch in tqdm(range(NUM_EPOCHS)):
    DANN_da.train(True)

    # iterate over the dataset
    for (source_images, source_labels), (target_images, _) in zip(src_dataloader, target_dataloader):
        source_images = source_images.to(DEVICE)
        target_images = target_images.to(DEVICE)
        source_labels = source_labels.to(DEVICE)

        # zero the gradient
        optimizer.zero_grad()

        # compute the classifier output for SOURCE images
        classifier_source_outputs = DANN_da(source_images, None)
        domain_classifier_source_outputs = DANN_da(source_images, epoch / NUM_EPOCHS)
        domain_classifier_target_outputs = DANN_da(target_images, epoch / NUM_EPOCHS)

        # compute the loss for the classifier
        c_loss = criterion(classifier_source_outputs, source_labels) # supervised task
        dc_loss = criterion(domain_classifier_source_outputs, torch.full((BATCH_SIZE, ), 1, device=DEVICE))
        dc_loss += criterion(domain_classifier_target_outputs, torch.full((BATCH_SIZE, ), 0, device=DEVICE))

        loss = c_loss + dc_loss
        # compute and propagate the gradient
        loss.backward()
        optimizer.step()

        wandb.log({"domain-adaptation-source-classifier-loss": c_loss})
        wandb.log({"domain-adaptation-source-domain-classifier-loss": dc_loss})
        wandb.log({"domain-adaptation-source-loss": dc_loss})

    target_loss = compute_classification_loss(DANN_da, target_dataloader)
    target_accuracy = compute_accuracy(DANN_da, target_dataloader)
    # print(f"[TARGET]: loss is {target_loss:.3f}, accuracy is {target_accuracy:.3f}")

    # at the end of each epoch compute the accuracy on the validation accuracy
    wandb.log({"domain-adaptation-target-accuracy": target_accuracy})
    wandb.log({"domain-adaptation-target-loss": target_loss})

    # Step the scheduler
    scheduler.step()

 28%|██▊       | 14/50 [06:43<17:16, 28.80s/it]


KeyboardInterrupt: ignored